In [1]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [2]:
# Prepare source dataset 

source_name = 'data/benchmark/Sboost_{sboost}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'count' : 100, 
    'shots' : 1000, 
    'sboost' : 0,
    'stepsize' : 0.25, 
    'accepterr': [0.01,0.05,0.1], 
    'iterations_limit' : 100, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'MA-PIA'
}

template = pd.read_csv('data/template_benchmark.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()

log_file, id = log._load_logfile()


Logfile data/benchmark/Sboost_0_ansatz_MA-PIA_stepsize_0.25_shots_1000.csv already exists


In [3]:


for i in tqdm.tqdm(range(Config['count']), desc = f"Generating matrices to solve with sboost {Config['sboost']}"):
    
    id += 1
    matrix = np.random.uniform(size = (Config['size'], Config['size']), low = -1, high = 1)
    matrix = (matrix.T + matrix) / 2
    current_datetime = datetime.now()
    data = { 
        'Id (int)' : id, 
        'Valid (int)' : 1, 
        'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
        'Matrix (mat)' : matrix,
        'Size (int)' : Config['size'],
        'Type (str)' : 'Benchmark',
        'Rank (int)' : -1,
        'Density (flt)' : -1.0,
        'Stiffness (flt)' : -1.0, 
        'Penalty (flt)' : -1.0,
        'Sboost (flt)' : Config['sboost'],
        'Test_solver (str)' : '-',
        'Test (flt)' : -1.0,
        'Test_state (vec)' : [],
        'Test_iterations (int)' : -1,
        'Test_time (flt)' : -1.0,
        'Train_solver (str)' : '-',
        'Preprocessing (str)' : '-',
        'Postprocessing (str)' : '-',
        'Train (flt)' : -1.0,
        'Train_state (vec)' : [],
        'Train_iterations (int)' : -1, 
        'Train_iterations_1 (int)' : -1,
        'Train_iterations_2 (int)' : -1,
        'Train_iterations_3 (int)' : -1,
        'Train_shots (int)': -1, 
        'Train_cost (flt)' : -1.0,
        'Train_time (flt)' : -1.0, 
        'Flags (str)' : '',
        'Baren (int)' : -1
    }
    
    log.log(data)


display(log.read(1))

Generating matrices to solve with sboost 0: 100%|██████████| 100/100 [00:00<00:00, 100.06it/s]


{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-09-02 19:11:59',
 'Matrix (mat)': array([[-0.39804876, -0.2445617 ,  0.33113785,  0.68077897,  0.44261496,
          0.16073465, -0.39471008,  0.16943005,  0.0548569 ,  0.10284181],
        [-0.2445617 ,  0.78084911, -0.017486  ,  0.01208866,  0.04770728,
          0.18876241,  0.16185143,  0.29214162, -0.15932606,  0.09824018],
        [ 0.33113785, -0.017486  ,  0.09558855, -0.15642884,  0.40169942,
         -0.69122928,  0.02489505, -0.25808543, -0.60462911,  0.3749415 ],
        [ 0.68077897,  0.01208866, -0.15642884,  0.91212647, -0.53576655,
         -0.30532393,  0.03663924, -0.83525742, -0.59573623, -0.24080271],
        [ 0.44261496,  0.04770728,  0.40169942, -0.53576655, -0.90016659,
         -0.10853992, -0.23674832,  0.35892358, -0.0294557 ,  0.27260358],
        [ 0.16073465,  0.18876241, -0.69122928, -0.30532393, -0.10853992,
         -0.2308474 , -0.40212481,  0.52499146, -0.79813303,  0.17836546],
        [-0.39

In [4]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])

In [5]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)

        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

Columns in data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other 

KeyboardInterrupt: 

In [6]:


ids = range (1,101)
cores = 25
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[  1,   2,   3,   4],
       [  5,   6,   7,   8],
       [  9,  10,  11,  12],
       [ 13,  14,  15,  16],
       [ 17,  18,  19,  20],
       [ 21,  22,  23,  24],
       [ 25,  26,  27,  28],
       [ 29,  30,  31,  32],
       [ 33,  34,  35,  36],
       [ 37,  38,  39,  40],
       [ 41,  42,  43,  44],
       [ 45,  46,  47,  48],
       [ 49,  50,  51,  52],
       [ 53,  54,  55,  56],
       [ 57,  58,  59,  60],
       [ 61,  62,  63,  64],
       [ 65,  66,  67,  68],
       [ 69,  70,  71,  72],
       [ 73,  74,  75,  76],
       [ 77,  78,  79,  80],
       [ 81,  82,  83,  84],
       [ 85,  86,  87,  88],
       [ 89,  90,  91,  92],
       [ 93,  94,  95,  96],
       [ 97,  98,  99, 100]])

{'Id (int)': 57, 'Valid (int)': 1, 'Updated (str)': '2025-09-02 19:11:59', 'Matrix (mat)': array([[-0.63093656,  0.2771309 , -0.0663414 ,  0.53597254,  0.14622855,
         0.06652688,  0.32009853,  0.14502843,  0.46954581,  0.79875542],
       [ 0.2771309 , -0.85665061, -0.6336539 ,  0.16662712, -0.13294179,
         0.50372653, -0.45176594, -0.06176712, -0.01902609, -0.46076118],
       [-0.0663414 , -0.6336539 ,  0.38845476, -0.26542929,  0.25836425,
         0.33686005,  0.28692515,  0.36000225,  0.25305258, -0.34163664],
       [ 0.53597254,  0.16662712, -0.26542929,  0.01296961,  0.22744814,
         0.2064502 , -0.72849102, -0.07089526, -0.58114188, -0.05596034],
       [ 0.14622855, -0.13294179,  0.25836425,  0.22744814,  0.6082555 ,
        -0.2483774 ,  0.2485427 ,  0.57583735,  0.40298347,  0.24020256],
       [ 0.06652688,  0.50372653,  0.33686005,  0.2064502 , -0.2483774 ,
         0.23549593, -0.27591122,  0.22106548,  0.53202738, -0.1295646 ],
       [ 0.32009853, -0.451

In [4]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = True, sboosted = False, sboost = line['Sboost (flt)'])
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        trained_data['Sboost (flt)'] = Config['sboost']
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [ ]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 40

solvers = [] 

depth = 3

template = pd.read_csv('data/template_benchmark.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)

ids = np.array(range(1, 601))
ids = ids.reshape(cores, len(ids) // cores) 

data = [[log.read(id) for id in subset] for subset in ids]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




201.75749893333418
13.26728774260160115.66673379050411645.37044019898791
7.89182673031007161.5832381487784617.90896676522833

47.1017749862046749.902914891811866
12.6389476298345866.9513859756487975



86.41643660202138
22.278848138352844179.90451875978816.4888210141499843.1870193403365267.14573945067219135.58310862728729
24.013299360876385

10.662719460779062
7.203499713671243





12.43231729137657230.9742090950751261.198214260265146.92014184762737
27.023125408751437
8.10630934211409189.58416318675151
14.938962271216239

18.18098851300619510.9587347502013325.0050756783787813.4305439868251814.214711988240378


12.128761028527855


7.7342268996256545




140.0925400555906
178.4232177164383
14.551748818192069
22.59788063854858
Ansatz: MA-PIA Iteration: 1 | State: [ 0.368  0.52   0.496  0.006 -0.042  0.194  0.042 -0.014  0.204 -0.032] | Sol: -3.8571895976483774 | Cursol: -1.2691539281669917 | Curenergy: -0.2105005376977297 | Preenergy: 0 | Baren_queue: [] | 100000000.0
Ansatz: MA-PIA Ite

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
